In [ ]:
import nltk
nltk.download("stopwords")
nltk.download('wordnet')
nltk.download('omw-1.4')
import re
import nltk
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
import re
import nltk
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
import pandas as pd
import numpy as np


import pandas as pd
import re
from datasets import DatasetDict

# Assuming you have a DataFrame named 'my_dataframe'

df = pd.read_csv('complete_dataset_labelled (1).csv')


df['text'] = df['text'].dropna()


t1=[]
for x in df['text']:
  y1=re.sub(r'[^\x00-\x7F]+',' ', x)
  t1.append(y1)

df['text']=t1


from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer


def removing_unwanted_data(text):
    # Preprocessing to remove unwanted characters and links
    text = re.sub(r'https?:\/\/.*[\r\n]*', '', text, flags=re.MULTILINE)
    text = re.sub(r'\<a href', ' ', text)
    text = re.sub(r'&amp;', '', text)
    text = re.sub(r'[_"\-;%()|+&=*%.,!?:#$@\[\]/]', ' ', text)
    text = re.sub(r'<br />', ' ', text)
    text = re.sub(r'\'', ' ', text)
    text = re.sub(r'\[[0-9]*\]', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'[^\x00-\x7F]+', ' ', text)
    text = re.sub("[^A-Z]", " ", text, 0, re.IGNORECASE)
    text = text.lower()
    text = re.sub(r'\d', ' ', text)
    text = re.sub(r'\s+', ' ', text)

    # Tokenization
    tokens = word_tokenize(text)

    # Remove Norwegian stop words
    norwegian_stopwords = set(stopwords.words('norwegian'))
    tokens = [word for word in tokens if word not in norwegian_stopwords]

    # Stemming
    stemmer = SnowballStemmer("norwegian")
    stemmed_tokens = [stemmer.stem(token) for token in tokens]

    # Remove words with less than 2 characters
    filtered_tokens = [token for token in stemmed_tokens if len(token) > 1]

    # Rejoin tokens into a single string
    text = ' '.join(filtered_tokens)

    return text

temp=df['text']
df['text']= list(map(removing_unwanted_data,df['text']))


# Assuming your original DataFrame looks like this: df["star_rating"] = [1, 2, 3, 4, 5, 1, 2, ...]
# Remap the values in the "star_rating" column

df["category"] = df["category"].replace({1: 0, 2: 1, 3: 2, 4: 3 , 5: 4})
df["category"].value_counts()


from keras.models import Sequential
from keras.layers import LSTM, Dense
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import fasttext

# Assuming df is your DataFrame with 'combined_text' and 'star_rating' columns
X_texts = df['text']
y = df['category']

# Train supervised fastText model
train_data = df.apply(lambda df: f"__label__{df['category']} {df['text']}", axis=1)
train_data.to_csv('train_data.txt', index=False, header=False)
model_fasttext = fasttext.train_supervised('train_data.txt', epoch=50, lr=0.1)

# Convert text to sequences
X_sequences = [model_fasttext.get_sentence_vector(text.replace('\n', '')) for text in X_texts]

# Pad sequences to ensure uniform length
X_padded = pad_sequences(X_sequences, dtype='float32')

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_padded, y, test_size=0.2, random_state=42)




#Save Model
import fasttext
from keras.models import Sequential, save_model, load_model


#model_fasttext.save_model("model_fasttext.bin")

# Load the model
loaded_model_fasttext = fasttext.load_model('model_fasttext.bin')


import numpy as np
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Bidirectional, LSTM, GRU, Dense,Dropout
from sklearn.model_selection import train_test_split

# Assuming X_padded is a 2D array, convert it to a 3D array
X_padded_3d = np.expand_dims(X_padded, axis=2)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_padded_3d, y, test_size=0.2, random_state=42)

# Define the model with both Bidirectional LSTM and Bidirectional GRU
model_bilstm_gru = Sequential()
model_bilstm_gru.add(Bidirectional(LSTM(100, return_sequences=True), input_shape=(X_padded_3d.shape[1], X_padded_3d.shape[2])))
model_bilstm_gru.add(Dropout(0.2))
model_bilstm_gru.add(Bidirectional(GRU(80, return_sequences=True)))
model_bilstm_gru.add(Bidirectional(LSTM(50, return_sequences=True)))  # Adjusted input shape
model_bilstm_gru.add(Bidirectional(GRU(30)))
model_bilstm_gru.add(Dense(5, activation='relu'))
model_bilstm_gru.add(Dense(5, activation='softmax'))

# Compile the model
model_bilstm_gru.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model and collect history for plotting
history_bilstm_gru = model_bilstm_gru.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)


# Plot the training and validation loss curves
plt.plot(history_bilstm_gru.history['loss'], label='Training Loss')
plt.plot(history_bilstm_gru.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss (Bidirectional GRU)')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

# Plot the training and validation accuracy curves
plt.plot(history_bilstm_gru.history['accuracy'], label='Training Accuracy')
plt.plot(history_bilstm_gru.history['val_accuracy'], label='Validation Accuracy')
plt.title('Training and Validation Accuracy (Bidirectional GRU)')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

# Evaluate the model
y_pred_proba = model_bilstm_gru.predict(X_test)
y_pred = [int(i.argmax()) for i in y_pred_proba]

from sklearn.metrics import roc_auc_score, roc_curve, auc
import matplotlib.pyplot as plt


from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import label_binarize

# Predict probabilities
y_pred_prob = model_bilstm_gru.predict(X_test)

# Binarize the y_test labels for multi-class ROC AUC calculation
y_test_binarized = label_binarize(y_test, classes=[0, 1, 2, 3, 4])

# Calculate the ROC-AUC score
roc_auc = roc_auc_score(y_test_binarized, y_pred_prob, multi_class='ovr')
print(f"ROC-AUC Score: {roc_auc}")


# Print accuracy and classification report
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

report = classification_report(y_test, y_pred)
print("Classification Report:\n", report)

# Assuming you know your class labels

import seaborn as sns  # Import seaborn for plotting
from sklearn.metrics import confusion_matrix


class_labels = ['1', '2', '3','4','5']  # Replace with your actual class names

# Generate and print the confusion matrix
cm = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:\n", cm)

# Plot the confusion matrix
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_labels, yticklabels=class_labels)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()